<a href="https://www.kaggle.com/code/scr0ll0/modeling-submission?scriptVersionId=154553923" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Imports/Downloads

In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler
from imblearn.under_sampling import RandomUnderSampler
from sklearn.linear_model import LogisticRegression, SGDClassifier

# Loading Data

In [2]:
train = pd.read_csv('/kaggle/input/daigt-v2-train-dataset/train_v2_drcat_02.csv')
test = pd.read_csv('/kaggle/input/llm-detect-ai-generated-text/test_essays.csv')
sample = pd.read_csv('/kaggle/input/llm-detect-ai-generated-text/sample_submission.csv')

# Cleaning Data

In [3]:
#Cleaning
train = train[['text', 'label']].copy()
sample['id'] = test['id']
test = test.drop(columns=['id', 'prompt_id'])

dev_x = train.drop(columns='label')
dev_y = train['label']

In [4]:
#Sampling
dev_x, dev_y = RandomUnderSampler(random_state=0).fit_resample(dev_x, dev_y)

In [5]:
#Vectorizing
vector = TfidfVectorizer(stop_words='english')
vector.fit(dev_x['text'])
dev_x = pd.DataFrame(vector.transform(dev_x['text']).toarray())
test = pd.DataFrame(vector.transform(test['text']).toarray())

In [6]:
#Scaling
scaler = StandardScaler()
scaler.fit(dev_x)
dev_x = scaler.transform(dev_x)
test = scaler.transform(test)

# Modeling

In [7]:
#Logistic Regression

#model = LogisticRegression()
#model.fit(dev_x, dev_y)

In [8]:
#SGD Classifier

model = SGDClassifier(loss='log_loss')
model.fit(dev_x, dev_y)

SGDClassifier(loss='log_loss')

# Predictions

In [9]:
sample['generated'] = model.predict_proba(test)[:, 1]

In [10]:
sample.to_csv('submission.csv', index=False)